## Run LISFLOOD with BMI sample model

In [ ]:
%matplotlib inline
from bmi.wrapper import BMIWrapper
import matplotlib.pyplot as plt
import numpy as np

BMI is all about interacting with the states, fluxes and parameters within runtime. So let's start by adding a bunch of water after one time step somewhere in the middle of our test domain, and see how that spreads in the first 100 days of simulation

In [ ]:
wrapper = BMIWrapper(engine="lisflood", configfile='/home/hcwinsemius/git/barotse/example_models/LFP_synthetic_2D/LFP_syntheticTestCase_2D.par')
wrapper.initialize()
# do one time step
wrapper.update()
# add a bunch of water in the middle (y-coord 250 is the middle)
h = wrapper.get_var('H')
h[250, :] += 3.
wrapper.get_var('H')[:] = h

The model domain looks like a small thin hillslope with a constant slope from the lower part to the upper part. Let's test the BMI functionality of lisflood here. We noticed that the timestep retrieved from .get_time_step is fixed on what is given in the .par file, whilst it should be updated dynamically. So we compute this from the per time step change. We also make a super simple plot of the water level outputs, aftger one day is passed, two days, 3, 4, 5 etc. and see if that gives expected results.

In [ ]:
t = 0 # is the moment in time that we want to plot something. After each plot we increase this by 86400 seconds (one day)
t1 = 0
p = 0  # plot number (let's do 100 plots for now)
volume = []
plt.figure(figsize=(20, 10))
while p < 100:
    # read current time
    t2 = wrapper.get_current_time()
    timestep = t2 - t1
    t1 = t2
    wrapper.update()
    if wrapper.get_current_time()> t:
        volume.append(wrapper.get_var('H').sum())
        # we passed t, so let's plot something
        print('Time step size is {:f} seconds'.format(timestep))
        p += 1 # we increase the plot number by one
        t += 86400 # we add one day to go to the next day
        plt.subplot(1, 100, p, frameon=False)
        plt.imshow(wrapper.get_var('H'), vmin=0., vmax=.8)
        plt.xticks([])
        plt.yticks([])
# 100 days have passed. We're done, so we close the model
wrapper.finalize()
# make a volume array and plot that as well


In [ ]:
f = plt.figure()
volume = np.array(volume)
plt.plot(volume)
plt.xlabel('time step [days]')
plt.ylabel('volumetric measure --')

so we clearly see the water moving through the domain. The upper side of the domain is the lowest part of the hillslope, so clearly that's where our 3 meters of water went to. We also see the upper boundaryu condition (bottom of graph) seeping through to the lower part of the domain. So great! we can interfere with the model in runtime! Let's see if the model can handle negative depth values. If not, we have to be careful not to get those!


In [ ]:
wrapper = BMIWrapper(engine="lisflood", configfile='/home/hcwinsemius/git/barotse/example_models/LFP_synthetic_2D/LFP_syntheticTestCase_2D.par')
wrapper.initialize()
wrapper.update()
# add a bunch of water in the middle (y-coord 250 is the middle)
h = wrapper.get_var('H')
h[250, :] -= 3.
wrapper.get_var('H')[:] = h
t = 0 # is the moment in time that we want to plot something. After each plot we increase this by 86400 seconds (one day)
t1 = 0
p = 0  # plot number (let's do 100 plots for now)
plt.figure(figsize=(20, 10))
while p < 100:
    # read current time
    t2 = wrapper.get_current_time()
    timestep = t2 - t1
    t1 = t2
    wrapper.update()
    if wrapper.get_current_time()> t:
        # we passed t, so let's plot something
        print('Time step size is {:f} seconds'.format(timestep))
        p += 1 # we increase the plot number by one
        t += 86400 # we add one day to go to the next day
        plt.subplot(1, 100, p, frameon=False)
        plt.imshow(wrapper.get_var('H'), vmin=0., vmax=.8)
        plt.xticks([])
        plt.yticks([])
# 100 days have passed. We're done, so we close the model
wrapper.finalize()


Ok, this looks good. Apparently the model will simply round depths to zero when negative values occur!